Referencing nuget packages

In [1]:
#r "nuget:XPlot.Plotly"
#r "nuget:Deedle"
#r "nuget:DotNetZip"
#r "nuget:NodaTime"

Installing package XPlot.Plotly..............done!

Successfully added reference to package XPlot.Plotly, version 3.0.1

Installing package Deedle....done!

Successfully added reference to package Deedle, version 2.1.0

Installing package DotNetZip....done!

Successfully added reference to package DotNetZip, version 1.13.4

Installing package NodaTime....done!

Successfully added reference to package NodaTime, version 2.4.7

**Type in password**

In [2]:
var password = "type password here";

In [3]:
using System.Linq;
using System.Collections.Generic;

(string, int[])[] SingleLineSource(string folder, int lineName) => new[] { (folder, new[] { lineName }), };
(string, int[])[] AllLinesSource() => 
    new[]
    {
        ("BadLines",       new[] { 490, 675, 677, 681, 671 } ),
        ("GoodLines",      new[] { 783, 834, 782, 837, 785 } ),
        ("InnerCityLines", new[] { 403, 505, 216, 906, 302 } ),
    };

**Decide if you want to play with a single line or all of them at once. Use corresponding line of code.**

In [4]:
//var sources = SingleLineSource("GoodLines", 837);
var sources = AllLinesSource();

Main code

In [5]:
using Deedle;
using Ionic.Zip;
using System.IO;
using System;
using System.Globalization;
using NodaTime;

var tz = DateTimeZoneProviders.Tzdb.GetZoneOrNull("Australia/Melbourne");

Frame<int, string> LoadData(string folder, string lineName)
{
    var zip = ZipFile.Read(string.Format(@"Data/Samples/{1}/Departures-{0}.zip", lineName, folder));
    zip.Password = password;
    using (var stream = zip.Entries.First().OpenReader())
    {
        var frame = 
            Frame
                .ReadCsv(stream)
                .Where(kvp => !string.IsNullOrEmpty(kvp.Value.GetAs<string>("ScheduledArrivalTime")));

        display($"Successfully read data: {frame.RowCount} records");

        var sa = frame.GetColumn<string>("ScheduledArrivalTime").Values.ToArray();
        var aa = frame.GetColumn<string>("ArrivalTime").Values.ToArray();
        var deviation = sa.Select((s, i) => (int)(DateTime.Parse(aa[i]) - DateTime.Parse(s)).TotalSeconds).ToArray();
        frame.AddColumn("Deviation", deviation);    
        var operatingDay = frame.GetColumn<DateTime>("OperatingDay").Values.Select(x => new LocalDate(x.Year, x.Month, x.Day)).ToArray();
        frame.AddColumn("OperatingDayNT", operatingDay);    
        var scheduledTimeZoned = frame.GetColumn<DateTime>("ScheduledArrivalTime").Values.Select(x => DateTime.SpecifyKind(x, DateTimeKind.Utc)).Select(Instant.FromDateTimeUtc).Select(x => x.InZone(tz)).ToArray();
        frame.AddColumn("ScheduledTimeZoned", scheduledTimeZoned);    

        return frame;
    }
}

In [6]:
public class StatsValues
{
    public double Min      { get; set; }
    public double Max      { get; set; }
    public double Mean     { get; set; }
    public double Median   { get; set; }
    public double StdDev   { get; set; }
}

public class NamedStatsValues : StatsValues
{
    public string LineName { get; set; }
    public string Title1   { get; set; }
    public string Title2   { get; set; }
    public string Title3   { get; set; }
    public string Title4   { get; set; }
}

NamedStatsValues CalcStats(Series<int, int> series, string lineName, string title1 = null, string title2 = null, string title3 = null, string title4 = null) => new NamedStatsValues 
{
    Min      = Math.Round(Stats.min(series)),
    Max      = Math.Round(Stats.max(series)),
    Mean     = Math.Round(Stats.mean(series)),
    StdDev   = Math.Round(Stats.stdDev(series)),
    Median   = Math.Round(Stats.median(series)),
    LineName = lineName,
    Title1   = title1,
    Title2   = title2,
    Title3   = title3,
    Title4   = title4,
};

In [7]:
var allFrames =
    sources
        .SelectMany(x => x.Item2.Select(ln => new { folder = x.Item1, lineName = ln.ToString() }))
        .ToArray()
        .Select(x => new { frame = LoadData(x.folder, x.lineName), x.folder, x.lineName })
        .ToDictionary(x => x.lineName);
display("All Loaded!");

Successfully read data: 16114 records

Successfully read data: 10756 records

Successfully read data: 27804 records

Successfully read data: 50754 records

Successfully read data: 54641 records

...

Successfully read data: 45695 records

Successfully read data: 507730 records

Successfully read data: 541946 records

Successfully read data: 304044 records

All Loaded!

In [8]:
var allStats = 
    allFrames
        .Values
        .Select(x => CalcStats(x.frame.GetColumn<int>("Deviation"), x.lineName, x.folder))
        .OrderBy(x => x.StdDev)
        .ToArray();
display(allStats);
var allStatsFrame = Frame.FromRecords(allStats);

index,LineName,Title1,Title2,Title3,Title4,Min,Max,Mean,Median,StdDev
0,837,GoodLines,<null>,<null>,<null>,-349,1780,71,46,145
1,677,BadLines,<null>,<null>,<null>,-359,3593,141,110,165
2,675,BadLines,<null>,<null>,<null>,-170,1712,140,105,196
3,834,GoodLines,<null>,<null>,<null>,-4722,5029,116,70,200
4,671,BadLines,<null>,<null>,<null>,-548,2556,147,94,214
5,681,BadLines,<null>,<null>,<null>,-4448,1887,91,45,224
6,783,GoodLines,<null>,<null>,<null>,-692,1815,137,124,228
7,785,GoodLines,<null>,<null>,<null>,-868,2353,175,127,230
8,302,InnerCityLines,<null>,<null>,<null>,-947,5146,107,65,241
9,490,BadLines,<null>,<null>,<null>,-881,2341,60,89,244


In [9]:
(int, double, double) CalcBuckets(Series<int, int> series)
{
    var stats = CalcStats(series, "", "");
    var bucketsNumber = 100;
    var bucketSize = (double)(stats.Max - stats.Min) / bucketsNumber;

    int getBucket(int value) => (int)(((double)value - stats.Min) / bucketSize);

    var distribution = series.GroupBy(kvp => getBucket(kvp.Value)).Select(kvp => kvp.Value.KeyCount);
    var maxBucket = distribution.Values.Max();    
    
    return (bucketsNumber, bucketSize, maxBucket);
}

Filtering function

In [10]:
static Frame<int, string> Filter(this Frame<int, string> frame, params Func<KeyValuePair<int, ObjectSeries<string>>, bool>[] conditions)
{
    var result = frame;
    foreach (var condition in conditions)
        result = result.Where(condition);

    return result;
}

In [11]:
using XPlot.Plotly;

PlotlyChart MakePlot(Series<int, int> series, string forWhat, int w, int h)
{
    var title = $"Deviation from Schedule (sec) for {forWhat}";
    
    var stats = CalcStats(series, "", "");
    var (bucketsNumber, bucketSize, maxBucket) = CalcBuckets(series);
    
    display(stats);
    
    var hist = 
        new Graph.Histogram
        {
            x = series.Values, 
            xbins   = new Graph.Xbins { start = stats.Min, end = stats.Max, size = bucketSize }, 
            marker  = new Graph.Marker { color = "yellow", line = new Graph.Line { color = "gray", width = 1 }},
            opacity = 0.75, 
            name = "Distribution"
        };

    var zero   = new Graph.Scatter { name = "Zero",   x = new[] { 0, 0 }, y = new[] { 0, maxBucket }, };
    var mean   = new Graph.Scatter { name = "Mean",   x = new[] { stats.Mean, stats.Mean }, y = new[] { 0, maxBucket }, };
    var median = new Graph.Scatter { name = "Median", x = new[] { stats.Median, stats.Median }, y = new[] { 0, maxBucket }, };

    var stdDev = 
        new Graph.Scatter
        {
            x = new[] { stats.Mean-stats.StdDev, stats.Mean-stats.StdDev, stats.Mean+stats.StdDev, stats.Mean+stats.StdDev },
            y = new[] { maxBucket, 0, 0, maxBucket },
            name = "StdDev",
        };

    var traces = new Graph.Trace[] { hist, mean, median, stdDev, zero };

    var plot = Chart.Plot(traces);  
    plot.WithXTitle("Deviation");
    plot.WithYTitle("Numner of arrivals");
    plot.WithTitle(title);
    plot.WithWidth(w);
    plot.WithHeight(h);
    return plot;
}

In [21]:
static LocalDate toLocalDate(this DateTime dt) => new LocalDate(dt.Year, dt.Month, dt.Day);

(Frame<int, string>, string) GetFrame(string lineName = null)
{
    var (frame, folder, lineName2) = 
        lineName != null 
        ? (allFrames[lineName].frame, allFrames[lineName].folder, allFrames[lineName].lineName)
        : (allFrames.First().Value.frame, allFrames.First().Value.folder, allFrames.First().Value.lineName);
    
    var title = $"{lineName2}, {folder}";
    
    return (frame, title);
}

(Frame<int, string>, string) PrepareFrame((Frame<int, string>, string) frameAndTitle, int? direction = null, int? stopCode = null, int? dayOfWeek = null, int? hrFrom = null, int? hrTo = null)
{
    var (frame, title) = frameAndTitle;
    var sw = System.Diagnostics.Stopwatch.StartNew();
    if (direction != null)
    {
        title += $" Direction {direction}";
        frame = frame.Filter(
            kvp => kvp.Value.GetAs<int>("Direction") == direction.Value);
    }
    if (stopCode != null)
    {
        title += $" StopCode {stopCode}";
        frame = frame.Filter(
            kvp => kvp.Value.GetAs<int>("IrsStopCode") == stopCode.Value);
    }
    if (dayOfWeek != null)
    {
        title += $" {(IsoDayOfWeek)dayOfWeek.Value}";
        frame = frame.Filter(
            kvp => kvp.Value.GetAs<LocalDate>("OperatingDayNT").DayOfWeek == (IsoDayOfWeek)(dayOfWeek.Value));
    }
    if (hrFrom != null)
    {
        title += $" {hrFrom} -";
        frame = frame.Filter(
            kvp => kvp.Value.GetAs<ZonedDateTime>("ScheduledTimeZoned").Hour >= hrFrom.Value);
    }
    if (hrTo != null)
    {
        title += $"- {hrTo}";
        frame = frame.Filter(
            kvp => kvp.Value.GetAs<ZonedDateTime>("ScheduledTimeZoned").Hour < hrTo.Value);
    }

    display($"Prepared in {sw.ElapsedMilliseconds} ms");
    return (frame, title.Trim());
}

PlotlyChart CreatePlot((Frame<int, string>, string) frameAndTitle, int? w = null, int? h = null)
{
    var (frame, title) = frameAndTitle;
    var deviationSeries = frame.GetColumn<int>("Deviation");
    var plot = MakePlot(deviationSeries, title, w ?? 800, h ?? 600);
    return plot;
}


**If no parameters specified, the first loaded line will be used**

**You can specify lineName, direction, stopCode for analysis plots**

**To see the list of available stopCodes use the last cell of the notebook**

In [22]:
display(CreatePlot(PrepareFrame(GetFrame())));


//display(CreatePlot(dayOfWeek: 6));
//display(CreatePlot("837"));
//display(CreatePlot(stopCode: 40866));

Prepared in 0 ms

LineName,Title1,Title2,Title3,Title4,Min,Max,Mean,Median,StdDev
,,<null>,<null>,<null>,-881,2341,60,89,244


In [24]:
var statsByDayOfWeek =
    allFrames.Keys
        .SelectMany(lineName => new[] { 1, 2, 3, 4, 5, 6, 7 }.SelectMany(dow => 
        {
            var (frame, title1) = GetFrame(lineName);
            var (frame2, title2) = PrepareFrame((frame, ""), dayOfWeek: dow);
            return new[] { 6, 10, 14, 18 }.Select(hrFrom =>
            {
                var (frame3, title3) = PrepareFrame((frame2, ""), hrFrom: hrFrom, hrTo: hrFrom+4);
                var deviationSeries = frame3.GetColumn<int>("Deviation");
                var stats = CalcStats(deviationSeries, lineName, title1, title2, title3);
                return stats;
            }).
            Where(x => !double.IsNaN(x.Min));
        }))
        .OrderBy(x => x.StdDev)
        .ToArray();

display(statsByDayOfWeek);

Prepared in 86 ms

Prepared in 56 ms

Prepared in 47 ms

Prepared in 56 ms

Prepared in 55 ms

...

Prepared in 301 ms

Prepared in 211 ms

Prepared in 213 ms

Prepared in 319 ms

index,LineName,Title1,Title2,Title3,Title4,Min,Max,Mean,Median,StdDev
0,671,"671, BadLines",Tuesday,6 -- 10,<null>,-69,183,74,74,50
1,681,"681, BadLines",Saturday,6 -- 10,<null>,-268,-48,-140,-131,51
2,671,"671, BadLines",Friday,6 -- 10,<null>,-129,204,61,53,54
3,677,"677, BadLines",Tuesday,6 -- 10,<null>,-104,205,71,72,56
4,834,"834, GoodLines",Saturday,14 -- 18,<null>,-135,196,23,18,60
5,675,"675, BadLines",Wednesday,6 -- 10,<null>,-110,94,-19,-24,65
6,671,"671, BadLines",Monday,6 -- 10,<null>,-42,293,105,92,71
7,675,"675, BadLines",Thursday,6 -- 10,<null>,-160,129,-11,-20,72
8,675,"675, BadLines",Tuesday,6 -- 10,<null>,-157,123,-7,-7,76
9,677,"677, BadLines",Wednesday,6 -- 10,<null>,-166,261,60,68,76


In [ ]:
void DisplayStops(string lineName = null)
{
    var frame = lineName == null ? allFrames.First().Value.frame : allFrames[lineName].frame;
    var stops = frame.GetColumn<string>("IrsStopCode").Values.Distinct().ToArray();
    display(stops);
}

In [ ]:
DisplayStops();
//DisplayStops("837");